In [2]:
import numpy as n
import pygame

pygame 2.5.2 (SDL 2.28.3, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [13]:
class Particle():
    def __init__(self, x, y, width, height, vx=0, vy=0):
        self.pos = pygame.Vector2(x, y)
        self.width = width
        self.height = height
        self.vel = pygame.Vector2(vx, vy)
        self.rect = pygame.Rect(self.pos.x, self.pos.y, self.width, self.height)
        
        self.updated = False
        
    def update(self):
        pass
    
    def collide(self, particle):
        pass

In [4]:
p = Particle(100, 100, 1, 1)
p.rect

<rect(100, 100, 1, 1)>

In [15]:
SCREEN_WIDTH = 800
SCREEN_HEIGHT = 800
NBOX_X = 10
NBOX_Y = 10
BOX_WIDTH = SCREEN_WIDTH // NBOX_X
BOX_HEIGHT = SCREEN_HEIGHT // NBOX_Y

class Box():
    def __init__(self, x, y, w, h, index):
        self.rect = pygame.Rect(x, y, w, h)
        self.index = index
        self.particles = []
        
    def isEdgeBox(self):
        return self.index % NBOX_X == 0 or self.index % NBOX_X == NBOX_X - 1 or self.index < NBOX_X or self.index >= NBOX_X * (NBOX_Y - 1)
        
    def updateBox(self):
        neighbors = self.getAdjBoxes()
        for particle in self.particles:
            self.wallCollide(particle)
            for neighbor in neighbors:
                for other_particle in neighbor.particles:
                    if particle != other_particle:
                        particle.update(other_particle)
                        if (particle.type == 'electron' or particle.type == 'proton') and particle.electroncapture:
                            x, y, velx, vely = particle.pos.x, particle.pos.y, particle.vel.x, particle.vel.y
                            x1, y1, velx1, vely1 = other_particle.pos.x, other_particle.pos.y, other_particle.vel.x, other_particle.vel.y
                            self.removeParticle(particle)
                            self.removeParticle(other_particle)
                            self.addParticle(Particle('neutron', x, y, velx, vely))
                            self.addParticle(Particle('neutrino', x1, y1, velx1, vely1))
            self.checkParticles()
        
    def getAdjBoxes(self):
        current = self.index
        right = self.index + 1
        left = self.index - 1
        up = self.index - NBOX_X
        down = self.index + NBOX_X
        up_right = up + 1
        up_left = up - 1
        down_right = down + 1
        down_left = down - 1
        
        if self.index % NBOX_X == 0:
            left = None
            up_left = None
            down_left = None
            
        if self.index % NBOX_X == NBOX_X - 1:
            right = None
            up_right = None
            down_right = None
            
        if self.index < NBOX_X:
            up = None
            up_right = None
            up_left = None
            
        if self.index >= NBOX_X * (NBOX_Y - 1):
            down = None
            down_right = None
            down_left = None
            
        adj_boxes = [boxes[i] for i in (current, right, left, up, down, up_right, up_left, down_right, down_left) if i is not None]
            
        return adj_boxes
    
    def isValidIndex(self, index):
        if not (0 <= index < len(boxes)):
            return False
        adj_indices = [self.index + 1, self.index - 1, self.index + NBOX_X, self.index - NBOX_X,
                       self.index + NBOX_X + 1, self.index + NBOX_X - 1, self.index - NBOX_X + 1, self.index - NBOX_X - 1]
        return index in adj_indices    

    def wallCollide(self, particle):
        if particle.rect.left < 0 or particle.rect.right > SCREEN_WIDTH:
            particle.vel.x = -particle.vel.x
            return True
    
        if particle.rect.top < 0 or particle.rect.bottom > SCREEN_HEIGHT:
            particle.vel.y = -particle.vel.y
            return True
        
        return False
    
    def checkParticles(self):
        gone_particles = []
        
        for particle in self.particles:
            box_x, box_y, box_w, box_h = self.rect
            
            if self.is_edge_box():
                collides = self.wallCollide(particle)
                    
            if not collides:
                if particle.rect.right > box_x + box_w and self.isValidIndex(self.index + 1):
                    boxes[self.index + 1].addParticle(particle)
                if particle.rect.left < box_x and self.isValidIndex(self.index - 1):
                    boxes[self.index - 1].addParticle(particle)
                if particle.rect.bottom > box_y + box_h and self.isValidIndex(self.index + NBOX_X):
                    boxes[self.index + NBOX_X].addParticle(particle)
                if particle.rect.top < box_y and self.isValidIndex(self.index - NBOX_X):
                    boxes[self.index - NBOX_X].addParticle(particle)
                if particle.rect.right > box_x + box_w and particle.rect.bottom > box_y + box_h and self.isValidIndex(self.index + NBOX_X + 1):
                    boxes[self.index + NBOX_X + 1].addParticle(particle)
                if particle.rect.right > box_x + box_w and particle.rect.top < box_y and self.isValidIndex(self.index - NBOX_X + 1):
                    boxes[self.index - NBOX_X + 1].addParticle(particle)
                if particle.rect.left < box_x and particle.rect.bottom > box_y + box_h and self.isValidIndex(self.index + NBOX_X - 1):
                    boxes[self.index + NBOX_X - 1].addParticle(particle)
                if particle.rect.left < box_x and particle.rect.top < box_y and self.isValidIndex(self.index - NBOX_X - 1):
                    boxes[self.index - NBOX_X - 1].addParticle(particle)
                if (particle.rect.left > box_x + box_w or particle.rect.right < box_x or particle.rect.bottom < box_y or particle.rect.top > box_y + box_h):
                    gone_particles.append(particle)
                    
        for particle in gone_particles:
            self.removeParticle(particle)
        
    def addParticle(self, particle):
        if not particle.updated and particle not in self.particles:
            self.particles.append(particle)
            particle.updated = True
        
    def removeParticle(self, particle):
        self.particles.remove(particle)
        particle.kill()
        
    def betaDecay(self)->bool:
        for particle in self.particles:
            if particle.type == 'neutron':
                x, y, velx, vely = particle.pos.x, particle.pos.y, particle.vel.x, particle.vel.y
                self.removeParticle(particle)
                boxes[self.index].addParticle(Particle('neutrino', x, y, velx, vely)) # to change... maybe
                boxes[self.index].addParticle(Particle('electron', x, y, -velx, -vely)) # to change... maybe
                player.respawn = False
                return True
        return False
            
    def contains_particles(self):
        return len(self.particles) > 0
        
    def draw(self, screen, color=(255, 255, 255), signal_content=False):
        if signal_content and self.contains_particles():
                color = (200, 120, 120)
        pygame.draw.rect(screen, color, self.rect)
        

boxes = []
for j in range(NBOX_Y):
    for i in range(NBOX_X):
        boxes.append(Box(i*BOX_WIDTH, j*BOX_HEIGHT, BOX_WIDTH, BOX_HEIGHT, i+j*NBOX_X))

In [ ]:
import pygame
import numpy as np
from forces import *
from player import *

SCREEN_WIDTH = 800
SCREEN_HEIGHT = 800
NBOX_X = 10
NBOX_Y = 10
BOX_WIDTH = SCREEN_WIDTH // NBOX_X
BOX_HEIGHT = SCREEN_HEIGHT // NBOX_Y

class Box():
    def __init__(self, x, y, w, h, index):
        self.rect = pygame.Rect(x, y, w, h)
        self.index = index
        self.particles = []
    
    def isEdgeBox(self):
        return self.index % NBOX_X == 0 or self.index % NBOX_X == NBOX_X - 1 or self.index < NBOX_X or self.index >= NBOX_X * (NBOX_Y - 1)
    
    def updateBox(self, player, dt, neutrons, neutrinos, board_width, board_height):
        neighbors = self.getAdjBoxes()
        for particle in self.particles:
            for neighbor in neighbors:
                for neighbor_particle in neighbor.particles:
                    self.handleCollision(particle, neighbor_particle)
                    force = particle.computeForce(neighbor_particle)
                    self.applyForce(particle, force)
            particle.update(player, self, dt, neutrons, neutrinos, board_width, board_height)
    
    def getAdjBoxes(self):
        right = self.index + 1
        left = self.index - 1
        up = self.index - NBOX_X
        down = self.index + NBOX_X
        up_right = up + 1
        up_left = up - 1
        down_right = down + 1
        down_left = down - 1
        
        if self.index % NBOX_X == 0:
            left = None
            up_left = None
            down_left = None
        
        if self.index % NBOX_X == NBOX_X - 1:
            right = None
            up_right = None
            down_right = None
        
        if self.index < NBOX_X:
            up = None
            up_right = None
            up_left = None
        
        if self.index >= NBOX_X * (NBOX_Y - 1):
            down = None
            down_right = None
            down_left = None
        
        adj_boxes = [boxes[i] for i in (right, left, up, down, up_right, up_left, down_right, down_left) if i is not None]
        return adj_boxes
    
    def wallCollide(self, particle):
        if particle.rect.left < 0 or particle.rect.right > SCREEN_WIDTH:
            particle.vel.x = -particle.vel.x
            return True
        if particle.rect.top < 0 or particle.rect.bottom > SCREEN_HEIGHT:
            particle.vel.y = -particle.vel.y
            return True
        return False
    
    def checkParticles(self, player, dt, neutrons, neutrinos, board_width, board_height):
        new_particles = []
        for particle in self.particles:
            box_x, box_y, box_w, box_h = self.rect
            collides = self.isEdgeBox() and self.wallCollide(particle)
            particle.update(player, self, dt, neutrons, neutrinos, board_width, board_height)

            if not collides:
                if particle.rect.right > box_x + box_w and self.isValidIndex(self.index + 1):
                    boxes[self.index + 1].addParticle(particle)
                if particle.rect.left < box_x and self.isValidIndex(self.index - 1):
                    boxes[self.index - 1].addParticle(particle)
                if particle.rect.bottom > box_y + box_h and self.isValidIndex(self.index + NBOX_X):
                    boxes[self.index + NBOX_X].addParticle(particle)
                if particle.rect.top < box_y and self.isValidIndex(self.index - NBOX_X):
                    boxes[self.index - NBOX_X].addParticle(particle)
                if particle.rect.right > box_x + box_w and particle.rect.bottom > box_y + box_h and self.isValidIndex(self.index + NBOX_X + 1):
                    boxes[self.index + NBOX_X + 1].addParticle(particle)
                if particle.rect.right > box_x + box_w and particle.rect.top < box_y and self.isValidIndex(self.index - NBOX_X + 1):
                    boxes[self.index - NBOX_X + 1].addParticle(particle)
                if particle.rect.left < box_x and particle.rect.bottom > box_y + box_h and self.isValidIndex(self.index + NBOX_X - 1):
                    boxes[self.index + NBOX_X - 1].addParticle(particle)
                if particle.rect.left < box_x and particle.rect.top < box_y and self.isValidIndex(self.index - NBOX_X - 1):
                    boxes[self.index - NBOX_X - 1].addParticle(particle)
                if (particle.rect.left > box_x + box_w or particle.rect.right < box_x or particle.rect.bottom < box_y or particle.rect.top > box_y + box_h):
                    new_particles.append(particle)
        for particle in new_particles:
            self.removeParticle(particle)
    
    def isValidIndex(self, index):
        return 0 <= index < len(boxes)

    def addParticle(self, particle):
        if not particle.updated and particle not in self.particles:
            self.particles.append(particle)
            particle.updated = True
    
    def removeParticle(self, particle):
        if particle in self.particles:
            self.particles.remove(particle)
    
    def containsParticles(self):
        return len(self.particles) > 0
    
    def draw(self, screen, color=(255, 255, 255), signal_content=False):
        if signal_content and self.containsParticles():
            color = (200, 120, 120)
        pygame.draw.rect(screen, color, self.rect)

    def checkCollisionsAndForces(self):
        neighbors = self.getAdjBoxes()
        for particle in self.particles:
            for neighbor in neighbors:
                for neighbor_particle in neighbor.particles:
                    if particle != neighbor_particle:
                        self.handleCollision(particle, neighbor_particle)
                        force = particle.computeForce(neighbor_particle)
                        self.applyForce(particle, force)

    @staticmethod
    def handleCollision(particle, neighbor_particle):
        dx = particle.pos.x - neighbor_particle.pos.x
        dy = particle.pos.y - neighbor_particle.pos.y
        distance = (dx ** 2 + dy ** 2) ** 0.5
        min_distance = particle.radius + neighbor_particle.radius

        if distance < min_distance:
            overlap = min_distance - distance
            particle.pos.x += (dx / distance) * overlap / 2
            particle.pos.y += (dy / distance) * overlap / 2
            neighbor_particle.pos.x -= (dx / distance) * overlap / 2
            neighbor_particle.pos.y -= (dy / distance) * overlap / 2

            particle.vel.x, neighbor_particle.vel.x = neighbor_particle.vel.x, particle.vel.x
            particle.vel.y, neighbor_particle.vel.y = neighbor_particle.vel.y, particle.vel.y

    @staticmethod
    def calculateForces(particle, neighbor_particle):
        force = particle.computeForce(neighbor_particle)
        return force

    @staticmethod
    def applyForce(particle, force):
        particle.acc += force / particle.mass

boxes = [Box(i * BOX_WIDTH, j * BOX_HEIGHT, BOX_WIDTH, BOX_HEIGHT, i + j * NBOX_X) for j in range(NBOX_Y) for i in range(NBOX_X)]


In [17]:
# starting pygame
pygame.init()

# setting up the Screen
width = 800
height = 600

# making the boxes
boxes = Boxes(width, height, 8, 6)

# making particle
p = Particle(95, 100, 20, 20)
p1 = Particle(399, 199, 60, 60)
boxes.addParticle(p)
boxes.addParticle(p1)

# setting up the screen
screen = pygame.display.set_mode((width, height))

# setting up the clock
clock = pygame.time.Clock()

# setting up the game loop
running = True
while running:
    screen.fill((0, 0, 0))
    
    boxes.draw(screen, signal_content=True)
    pygame.draw.rect(screen, (250,250,250), p.rect)
    pygame.draw.rect(screen, (250,250,250), p1.rect)
        
    
    
    
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
    
    pygame.display.flip()
    clock.tick(60)

TypeError: Box.__init__() missing 1 required positional argument: 'index'

: 